## _*FermionicOperator and qubit mapping*_

When we compute a FermionicOperator in Qiskit Chemistry it needs to be converted to a qubit operator to run on the simulator or real device. The FermionicOperator is built from electron integrals where electrons behave anti-symmetrically under swap. Qubits however do not exhibit this behavior and hence a mapping is needed to ensure that this is accounted for.

Here we have the jordan wigner mapping, the bravyi-kitaev mapping and a parity.

In [3]:
import numpy as np

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

from qiskit_algorithms import NumPyMinimumEigensolver

from qiskit_algorithms.optimizers import SLSQP

from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_algorithms import VQE
from qiskit.primitives import Estimator

In [19]:
driver = PySCFDriver(atom='H .0 .0 .0; H .0 .0 0.74279')
molecule = driver.run()

mappers = [
    ('Jordan-Wigner', JordanWignerMapper()),
    ('Parity', ParityMapper(num_particles=molecule.num_particles)),
    ('Bravyi-Kitaev', BravyiKitaevMapper())
]

In [20]:
transformer = FreezeCoreTransformer()
molecule = transformer.transform(molecule)
hamiltonian = molecule.hamiltonian.second_q_op()

for name, mapper in mappers:
    print(f'{name}:')

    qubit_op = mapper.map(hamiltonian)
    print(f'--- Number of qubits needed: {qubit_op.num_qubits}, Number of paulis: {qubit_op.size}')

    algo = NumPyMinimumEigensolver()
    algo.filter_criterion = molecule.get_default_filter_criterion()
    solver = GroundStateEigensolver(mapper, algo)
    result = solver.solve(molecule)
    print(f'--- NumPyMinimumEigensolver result: {result.total_energies[0]} Ha')

    optimizer = SLSQP(maxiter=10000, ftol=1e-9)
    ansatz = UCCSD(
        molecule.num_spatial_orbitals,
        molecule.num_particles,
        mapper,
        initial_state=HartreeFock(
            molecule.num_spatial_orbitals,
            molecule.num_particles,
            mapper,
        ),
    )
    vqe = VQE(Estimator(), ansatz, optimizer)
    vqe.initial_point = [0] * ansatz.num_parameters
    algo = GroundStateEigensolver(mapper, vqe)
    result = algo.solve(molecule)
    print(f'--- VQE result: {result.total_energies[0]} Ha')

Jordan-Wigner:
--- Number of qubits needed: 4, Number of paulis: 15
--- NumPyMinimumEigensolver result: -1.1372534438864932 Ha
--- VQE result: -1.1372534438281199 Ha
Parity:
--- Number of qubits needed: 2, Number of paulis: 5
--- NumPyMinimumEigensolver result: -1.1372534438864927 Ha
--- VQE result: -1.13725344382852 Ha
Bravyi-Kitaev:
--- Number of qubits needed: 4, Number of paulis: 15
--- NumPyMinimumEigensolver result: -1.1372534438864927 Ha
--- VQE result: -1.137253443827503 Ha


In [21]:
! pip freeze | grep qiskit

qiskit==1.1.0
qiskit-aer==0.14.2
qiskit-algorithms==0.3.0
qiskit-ibm-runtime==0.25.0
qiskit-machine-learning==0.7.2
qiskit-nature==0.7.2
qiskit-nature-pyscf==0.4.0
qiskit-qasm3-import==0.5.0
qiskit-transpiler-service==0.4.5
